In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data_final.csv',encoding='utf-8',index_col=0)

In [ ]:
#모델에 쓰지 못할 데이터 제거
df_cleaned = df.dropna(subset=['총구매수'])
df_cleaned = df_cleaned.dropna(subset=['총조회수'])
df_cleaned.loc[df_cleaned['총구매수'].isnull()==True]

In [ ]:
#사이즈 수가 0인것을 제거
df_cleaned.loc[df_cleaned['사이즈 수']==0]

rows_with_zero_size_count = df_cleaned.loc[df_cleaned['사이즈 수'] == 0]

b = df_cleaned.drop(index=rows_with_zero_size_count.index)

In [ ]:
# 브랜드별 주요 구매 고객 선정
p_list = ['18이하_구매수_비율','19~23_구매수_비율','24~28_구매수_비율','29~33_구매수_비율','34~39_구매수_비율','40이상_구매수_비율']
v_list = ['18이하_조회수_비율','19~23_조회수_비율','24~28_조회수_비율','29~33_조회수_비율','34~39_조회수_비율','40이상_조회수_비율']
na_list=[18,21,26,31,35,40]
for i in range(len(df_cleaned)):
    print(i)
    a_list=[]
    b_list=[]
    for j in range(len(p_list)):
        print(int(df_cleaned.at[i,p_list[j]]))
        print(int(df_cleaned.at[i,v_list[j]]))
        a_list.append(df_cleaned.at[i,p_list[j]])
        b_list.append(df_cleaned.at[i,v_list[j]])
    df_cleaned.at[i,'주요구매연령']= na_list[a_list.index(max(a_list))]
    df_cleaned.at[i,'주요열람연령']= na_list[b_list.index(max(b_list))]

#주요 구매 연령 선정
grouped = df_cleaned.groupby('브랜드')
result = grouped['주요구매연령'].max()
df3 = result.reset_index()
df3.columns = ['브랜드', '브랜드_주요_구매_연령']

#주요 조회 연령 선정
grouped1 = df_cleaned.groupby('브랜드')
result1 = grouped1['주요열람연령'].max()
df4 = result1.reset_index()
df4.columns = ['브랜드', '브랜드_주요_조회_연령']





In [ ]:
#브랜드별 주요 고객 성별 선정
p_list = ['남자_구매수','여자_구매수']
v_list = ['남자_조회수','여자_조회수']
na_list=['남자','여자']
for i in range(len(df_cleaned)):
    print(i)
    a_list=[]
    b_list=[]
    for j in range(len(p_list)):
        print(int(df_cleaned.at[i,p_list[j]]))
        print(int(df_cleaned.at[i,v_list[j]]))
        a_list.append(df_cleaned.at[i,p_list[j]])
        b_list.append(df_cleaned.at[i,v_list[j]])
    df_cleaned.at[i,'주요구매성별']= na_list[a_list.index(max(a_list))]
    df_cleaned.at[i,'주요열람성별']= na_list[b_list.index(max(b_list))]
#주요구매 성별
grouped3 = df_cleaned.groupby('브랜드')
result3 = grouped3['주요구매성별'].max()
df5 = result3.reset_index()
df5.columns = ['브랜드', '브랜드_주요_구매_성별']
#주요 열람 성별
grouped4 = df_cleaned.groupby('브랜드')
result4 = grouped4['주요열람성별'].max()
df6 = result4.reset_index()
df6.columns = ['브랜드', '브랜드_주요_조회_성별']

In [ ]:
df = pd.read_csv('/content/MyDrive/MyDrive/csv_file/origin_data1.csv',encoding='utf-8',index_col=0)
df_ml=pd.read_csv('/content/MyDrive/MyDrive/csv_file/ml_data_final2.csv',encoding='utf-8',index_col=0)

In [ ]:
#RGB값 -> 핵사코드 -> 10진수
#rgb->핵사
def rgb_to_hex(r, g, b):
    """
    RGB 값을 16진수 문자열로 변환합니다'.
    """
    # 각 값을 16진수로 변환하고, 두 자리 변환
    hex_value = '{:02X}{:02X}{:02X}'.format(r, g, b)
    return hex_value
#핵사-> 10진수
def hex_to_number(hex_str):
    """
    16진수 문자열을 숫자로 변환합니다.
    """
    # 16진수 문자열을 10진수 숫자로 변환
    number = int(hex_str, 16)
    return number
#두개를 종합
def rgb_to_hex_and_decimal(r, g, b):
    hex_value = rgb_to_hex(r, g, b)
    number =  hex_to_number(hex_value)
    return number

In [ ]:
#새로운 컬럼 생성
df['rgb_1']=0
df['rgb_2']=0
#rgb값을1 10진수로 할당
for idx,i in enumerate(df['색상_rgb']):
  rgb_list = list(map(int, i.split(',')))
  #rgb값이 2개이면 서로 다른 색상 배치
  if len(rgb_list) >3:
    number = rgb_to_hex_and_decimal(rgb_list[0], rgb_list[1], rgb_list[2])
    number2 = rgb_to_hex_and_decimal(rgb_list[3], rgb_list[4], rgb_list[5])

  else:
    number = rgb_to_hex_and_decimal(rgb_list[0], rgb_list[1], rgb_list[2])
    number2 = number
  print(number ,number2)
  df.at[idx,'rgb_1'] = int(number)
  df.at[idx,'rgb_2'] = int(number2)

In [ ]:
# 브랜드별로 그룹화 통계 코드 -> 해당 name을 바꿔가며 리뷰수,총구매수,조회수등을 할당
grouped = df.groupby('브랜드')
name = '총구매수'
# 각 통계치 계산
average_reviews = grouped[name].mean()
max_reviews = grouped[name].max()
min_reviews = grouped[name].min()
median_reviews = grouped[name].median()
variance_reviews = grouped[name].var()
sum_reviews = grouped[name].sum()

result3 = pd.DataFrame({
    f'최대_{name}': max_reviews,
    f'최소_{name}': min_reviews,
    f'중앙값_{name}': median_reviews,
    #'리뷰 수 분산': variance_reviews,
    #'리뷰 수 합계': sum_reviews
})

print(result3)

In [ ]:
#라벨 인코딩
import pandas as pd
from sklearn.preprocessing import LabelEncoder



# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

#컬럼에 라벨 인코딩 적용
merged_df1['브랜드_구매_성별'] = label_encoder.fit_transform(merged_df1['브랜드_주요_구매_성별'])
merged_df1['브랜드_조회_성별'] =  label_encoder.fit_transform(merged_df1['브랜드_주요_조회_성별'])
merged_df1['브랜드_라벨링'] =  label_encoder.fit_transform(merged_df1['브랜드'])

In [ ]:
#결측치 제거 및 자료형 변환
df_ml = df_ml.loc[df_ml['브랜드_구매_성별'].isnull()==False]
df_ml = df_ml.fillna(0)
# 변환할 컬럼 리스트
columns_to_convert = ['제품코드', '원가', '할인가', '할인율', '사이즈 수', 'human_ditect',
       '컬러_encoded', '스타일샷_유무', '총구매수', '총조회수_평균', '총구매수_평균',
       '판매량등급', '브랜드_구매_성별', '브랜드_조회_성별', 'rgb_1', 'rgb_2', '리뷰_평균',
       'word1_score', 'word2_score', 'word3_score', '브랜드 좋아요 수', '브랜드_라벨링']
# 여러 컬럼을 float으로 변환
df_ml[columns_to_convert] = df_ml[columns_to_convert].astype(float)